In [1108]:
# This notebook will take the housing data scraped earlier and clean/format it for further investigation.
import pandas as pd
import numpy as np
import csv

pd.options.display.max_columns = None

In [1109]:
priceRange = [
    [1, 250000], [250001, 300000], [300001, 350000], [350001, 400000],
    [400001, 450000], [450001, 500000], [500001, 750000], [750001, 1000000],
    [1000001, 2000000], [2000001, 5000000], [5000001, 50000000]
    ]

In [1110]:
# Create function to take in column of dataframe and return modified column
# Function will take in resoFacts.associationFee and determine if its monthly, quarterly, etc and div value by appropriate amount
# Returned dataframe series will just a number representing monthly HOA fee due.

def hoaFees(series: pd.DataFrame):
    paymentFreq = series.apply(lambda x: str(x).strip("$,1234567890 "))
    payment = series.apply(lambda x: str(x).strip("$,-abcdefghijklmnopqrstuvwxyz ").replace(',', ''))
    payment = payment.apply(lambda x: float(x))

    paymentFreq.replace('', 0, inplace=True)

    for i, freq in enumerate(paymentFreq):
        if freq == 'monthly':
            payment[i] = payment[i]

        elif freq == 'quarterly':
            payment[i] = payment[i]/4

        elif freq == 'semi-annually':
            payment[i] = payment[i]/6

        elif freq == 'annually':
            payment[i] = payment[i]/12

        else:
            payment[i] = 0
    
    payment = round(payment)
    return payment

# Concat partial URL with website prefix
def fullURL(hdpURL: str):
    hdpURL = 'www.zillow.com' + str(hdpURL)
    return hdpURL

# Set area units to a consistent value of 'Square Feet'
def areaUnitConversion(units: pd.Series):
    for idx in units.index:
        if units.lotAreaUnits[idx] == 'Acres':
            units.lotAreaValue[idx] = units.lotAreaValue[idx] * 43560
            units.lotAreaUnits[idx] = 'Square Feet'

        elif units.lotAreaUnits[idx] == 'sqft':
            units.lotAreaUnits[idx] = 'Square Feet'

    return units

In [1111]:
for i, price in enumerate(priceRange):
    if i == 0:
        df_forSale = pd.read_csv(f'Raw_Housing_Data/housingData_{price[0]}_{price[1]}.csv',
                                 index_col=0, dtype={'zipcode':str})
        df_recentlySold = pd.read_csv(f'Raw_Housing_Data/housingData_recentlySold_{price[0]}_{price[1]}.csv',
                                      index_col=0, dtype={'zipcode':str})
    else:
        df_forSale = pd.concat([df_forSale, pd.read_csv(f'Raw_Housing_Data/housingData_{price[0]}_{price[1]}.csv',
                                                        index_col=0)], axis=0, ignore_index=True)
        df_recentlySold = pd.concat([df_recentlySold, pd.read_csv(f'Raw_Housing_Data/housingData_recentlySold_{price[0]}_{price[1]}.csv',
                                                                  index_col=0)], axis=0, ignore_index=True)

In [1112]:
columnNames_forSale = list(map(lambda x: x.split(', '), df_forSale.columns))
columnNames_recentlySold = list(map(lambda x: x.split(','), df_recentlySold))

with open('Raw_Housing_Data/dataframeColumnNames_forSale.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(columnNames_forSale)

with open('Raw_Housing_Data/dataframeColumnNames_recentlySold.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(columnNames_recentlySold)

In [1113]:
# Manually select column names I want to keep in my downsized data frame
columnNames_forSale = ['zpid','homeType','homeStatus','timeOnZillow.1','price','streetAddress','zipcode','yearBuilt','bedrooms','bathrooms',
                       'livingAreaValue','livingAreaUnits','lotAreaValue','lotAreaUnits','zestimate','latitude','longitude',
                       'favoriteCount','pageViewCount',
                       'newConstructionType','resoFacts.associationFee','listingSubType.isForeclosure','resoFacts.isSeniorCommunity','resoFacts.propertySubType','hdpUrl']

columnNames_recentlySold = ['zpid','homeType','homeStatus','dateSoldString','timeOnZillow.1','price','streetAddress','zipcode','yearBuilt','bathrooms','bedrooms',
                            'livingAreaValue','livingAreaUnits','lotAreaValue','lotAreaUnits','zestimate','latitude','longitude',
                            'favoriteCount','pageViewCount',
                            'newConstructionType','resoFacts.associationFee','listingSubType.isForeclosure','resoFacts.isSeniorCommunity', 'resoFacts.propertySubType','hdpUrl']

df_forSale_trunc = df_forSale.loc[:,columnNames_forSale]
df_recentlySold_trunc = df_recentlySold.loc[:,columnNames_recentlySold]
# del df_forSale, df_recentlySold

In [1114]:
# Do some initial data cleanup
# Set NaN values to 0. Compute quivalent monthly HOA fees to keep data consistent.
df_forSale_trunc['resoFacts.associationFee'] = df_forSale_trunc['resoFacts.associationFee'].fillna(0)
df_recentlySold_trunc['resoFacts.associationFee'] = df_recentlySold_trunc['resoFacts.associationFee'].fillna(0)
df_forSale_trunc['resoFacts.associationFee'] = hoaFees(df_forSale_trunc['resoFacts.associationFee'])
df_recentlySold_trunc['resoFacts.associationFee'] = hoaFees(df_recentlySold_trunc['resoFacts.associationFee'])

# Set NaN values to 'False'. True is already designated by default, where applicable.
df_forSale_trunc['resoFacts.isSeniorCommunity'] = df_forSale_trunc['resoFacts.isSeniorCommunity'].fillna(False)
df_recentlySold_trunc['resoFacts.isSeniorCommunity'] = df_recentlySold_trunc['resoFacts.isSeniorCommunity'].fillna(False)

# Reformat the string representing the property subtype so it can be easily worked on.
# Removing any RV Community data as it is not a valid 'single family home' sub type for this scope.
df_forSale_trunc['resoFacts.propertySubType'] = df_forSale_trunc['resoFacts.propertySubType'].apply(lambda x: str(x).strip("['']"))
df_recentlySold_trunc['resoFacts.propertySubType'] = df_recentlySold_trunc['resoFacts.propertySubType'].apply(lambda x: str(x).strip("['']"))
df_forSale_trunc = df_forSale_trunc.loc[df_forSale_trunc['resoFacts.propertySubType']!='RV Community']
df_recentlySold_trunc = df_recentlySold_trunc.loc[df_recentlySold_trunc['resoFacts.propertySubType']!='RV Community']

# Add prefix to website
df_forSale_trunc['hdpUrl']=df_forSale_trunc['hdpUrl'].apply(fullURL)
df_recentlySold_trunc['hdpUrl']=df_recentlySold_trunc['hdpUrl'].apply(fullURL)

# Convert yearBuilt to integer and drop any rows that have yearBuilt as NaN
df_forSale_trunc.dropna(subset=['yearBuilt'], inplace=True)
df_recentlySold_trunc.dropna(subset=['yearBuilt'], inplace=True)
df_forSale_trunc['yearBuilt']=df_forSale_trunc['yearBuilt'].astype(int)
df_recentlySold_trunc['yearBuilt']=df_recentlySold_trunc['yearBuilt'].astype(int)

# Drop all labels with lotAreaValue = NaN
df_forSale_trunc.dropna(subset=['lotAreaValue'], inplace=True)
df_recentlySold_trunc.dropna(subset=['lotAreaValue'], inplace=True)

# Drop all labels with lat/long = NaN
df_forSale_trunc.dropna(subset=['latitude', 'longitude'], inplace=True)
df_recentlySold_trunc.dropna(subset=['latitude', 'longitude'], inplace=True)

# Fill all zestimate = NaN values with 0
value = {'zestimate':0}
df_forSale_trunc.fillna(value=value, inplace=True)
df_recentlySold_trunc.fillna(value=value, inplace=True)

# Drop all labels with livingAreaValue = NaN -- This is the primary feature for price prediction.
# Could try to do more searches on $/sqft but for now this will suffice.
df_forSale_trunc.dropna(subset=['livingAreaValue'], inplace=True)
df_recentlySold_trunc.dropna(subset=['livingAreaValue'], inplace=True)

# Drop all bathroom and bedroom values of NaN
df_forSale_trunc.dropna(subset=['bathrooms', 'bedrooms'], inplace=True)
df_recentlySold_trunc.dropna(subset=['bathrooms', 'bedrooms'], inplace=True)

# Fill favoriteCount and pageViewCount with 0
value = {'favoriteCount':0, 'pageViewCount':0}
df_forSale_trunc.fillna(value=value, inplace=True)
df_recentlySold_trunc.fillna(value=value, inplace=True)

# Need to set all timeOnZillow.1 values of less than one day to one day.
df_recentlySold_trunc['timeOnZillow.1'].mask(df_recentlySold_trunc['dateSoldString']=='2024-02-15', '1 day', inplace=True)

# Need to set newConstructionType to either True of False
df_recentlySold_trunc['newConstructionType'] = df_recentlySold_trunc['newConstructionType'].apply(lambda x: str(x))
df_recentlySold_trunc['newConstructionType'] = df_recentlySold_trunc['newConstructionType']!='nan'

# Need to keep lotAreaUnits and livingAreaUnits consistently set to Square Feet.
# Will need to create a function to deal with these.
temp = df_forSale_trunc[['lotAreaValue','lotAreaUnits']]
temp = areaUnitConversion(temp)
# df_forSale_trunc[['lotAreaValue','lotAreaUnits']] = areaUnitConversion(temp)
# temp = df_recentlySold_trunc[['lotAreaValue','lotAreaUnits']]
# df_recentlySold_trunc[['lotAreaValue','lotAreaUnits']] = areaUnitConversion(temp)
del temp

/var/folders/nh/hckbtchx5mdb0y_ryrrfbzxm0000gn/T/ipykernel_96006/419168942.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  units.lotAreaValue[idx] = units.lotAreaValue[idx] * 43560
/var/folders/nh/hckbtchx5mdb0y_ryrrfbzxm0000gn/T/ipykernel_96006/419168942.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  units.lotAreaUnits[idx] = 'Square Feet'


In [1115]:
# Reset Index
df_forSale_trunc.reset_index(inplace=True)
df_forSale_trunc.drop(columns=['index'], inplace=True)

df_recentlySold_trunc.reset_index(inplace=True)
df_recentlySold_trunc.drop(columns=['index'], inplace=True)

In [1116]:
df_forSale_trunc.describe(include='all')

,zpid,homeType,homeStatus,timeOnZillow.1,price,streetAddress,zipcode,yearBuilt,bedrooms,bathrooms,livingAreaValue,livingAreaUnits,lotAreaValue,lotAreaUnits,zestimate,latitude,longitude,favoriteCount,pageViewCount,newConstructionType,resoFacts.associationFee,listingSubType.isForeclosure,resoFacts.isSeniorCommunity,resoFacts.propertySubType,hdpUrl
count,2.245000e+03,2245,2245,2245,2.245000e+03,2245,2245.0,2245.000000,2245.000000,2245.000000,2245.000000,2245,2245.000000,2245,2.245000e+03,2245.000000,2245.000000,2245.000000,2245.000000,376,2245.000000,2245,2245,2245,2245
unique,NaN,1,1,283,NaN,2242,42.0,NaN,NaN,NaN,NaN,1,NaN,2,NaN,NaN,NaN,NaN,NaN,1,NaN,2,2,7,2245
top,NaN,SINGLE_FAMILY,FOR_SALE,3 days,NaN,990 Quaker St,32909.0,NaN,NaN,NaN,NaN,Square Feet,NaN,Square Feet,NaN,NaN,NaN,NaN,NaN,BUILDER_SPEC,NaN,False,False,Single Family Residence,www.zillow.com/homedetails/5650-Stamford-St-Mi...
freq,NaN,2245,2245,66,NaN,2,325.0,NaN,NaN,NaN,NaN,2245,NaN,1498,NaN,NaN,NaN,NaN,NaN,376,NaN,2230,2185,2234,1
mean,2.383089e+08,NaN,NaN,NaN,5.675352e+05,NaN,NaN,1995.680624,3.516258,2.437862,2009.595100,NaN,5619.762218,NaN,3.637318e+05,28.173165,-80.686566,47.288196,866.015590,NaN,34.018263,NaN,NaN,NaN,NaN
std,5.211176e+08,NaN,NaN,NaN,5.394611e+05,NaN,NaN,23.921179,0.787224,0.881222,807.331728,NaN,4250.178588,NaN,4.673281e+05,0.222370,0.081591,55.133603,891.594184,NaN,84.445313,NaN,NaN,NaN,NaN
min,4.336783e+07,NaN,NaN,NaN,7.500000e+04,NaN,NaN,1884.000000,0.000000,1.000000,160.000000,NaN,0.000000,NaN,0.000000e+00,27.833658,-80.930824,0.000000,8.000000,NaN,0.000000,NaN,NaN,NaN,NaN
25%,4.345368e+07,NaN,NaN,NaN,3.349990e+05,NaN,NaN,1979.000000,3.000000,2.000000,1536.000000,NaN,0.730000,NaN,0.000000e+00,27.989408,-80.729880,14.000000,287.000000,NaN,0.000000,NaN,NaN,NaN,NaN
50%,5.461239e+07,NaN,NaN,NaN,3.999000e+05,NaN,NaN,1999.000000,3.000000,2.000000,1865.000000,NaN,7405.200000,NaN,3.256000e+05,28.121946,-80.679330,32.000000,653.000000,NaN,0.000000,NaN,NaN,NaN,NaN
75%,1.041322e+08,NaN,NaN,NaN,5.999000e+05,NaN,NaN,2021.000000,4.000000,3.000000,2271.000000,NaN,10018.800000,NaN,4.429000e+05,28.340923,-80.638830,61.000000,1149.000000,NaN,33.000000,NaN,NaN,NaN,NaN


In [1117]:
df_recentlySold_trunc.describe(include='all')

,zpid,homeType,homeStatus,dateSoldString,timeOnZillow.1,price,streetAddress,zipcode,yearBuilt,bathrooms,bedrooms,livingAreaValue,livingAreaUnits,lotAreaValue,lotAreaUnits,zestimate,latitude,longitude,favoriteCount,pageViewCount,newConstructionType,resoFacts.associationFee,listingSubType.isForeclosure,resoFacts.isSeniorCommunity,resoFacts.propertySubType,hdpUrl
count,5.682000e+03,5682,5682,5682,5682,5.682000e+03,5682,5682.0,5682.000000,5682.000000,5682.000000,5682.000000,5682,5682.000000,5682,5.682000e+03,5682.000000,5682.000000,5682.000000,5682.000000,5682,5682.000000,5682,5682,5682,5682
unique,NaN,1,2,286,327,NaN,5667,51.0,NaN,NaN,NaN,NaN,1,NaN,3,NaN,NaN,NaN,NaN,NaN,2,NaN,2,2,7,5682
top,NaN,SINGLE_FAMILY,RECENTLY_SOLD,2023-12-29,76 days,NaN,2922 Tidepool Pl,32940.0,NaN,NaN,NaN,NaN,Square Feet,NaN,Square Feet,NaN,NaN,NaN,NaN,NaN,False,NaN,False,False,Single Family Residence,www.zillow.com/homedetails/1475-Leeward-Ave-SE...
freq,NaN,5682,5667,101,89,NaN,2,660.0,NaN,NaN,NaN,NaN,5682,NaN,3740,NaN,NaN,NaN,NaN,NaN,5677,NaN,5656,5493,5160,1
mean,2.282110e+08,NaN,NaN,NaN,NaN,4.900480e+05,NaN,NaN,1995.138684,2.367582,3.455121,1975.512496,NaN,5465.043544,NaN,4.848977e+05,28.187325,-80.689312,2.289335,58.419395,NaN,40.037311,NaN,NaN,NaN,NaN
std,5.000679e+08,NaN,NaN,NaN,NaN,3.881326e+05,NaN,NaN,22.871024,0.811598,0.764815,858.913839,NaN,4170.162272,NaN,3.854989e+05,0.207348,0.079472,14.691526,253.115132,NaN,98.398632,NaN,NaN,NaN,NaN
min,4.336795e+07,NaN,NaN,NaN,NaN,2.200000e+02,NaN,NaN,1901.000000,0.000000,0.000000,0.000000,NaN,0.000000,NaN,0.000000e+00,27.833752,-80.928590,0.000000,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN
25%,4.344606e+07,NaN,NaN,NaN,NaN,3.029900e+05,NaN,NaN,1979.000000,2.000000,3.000000,1492.000000,NaN,0.570000,NaN,3.026000e+05,28.012663,-80.733130,0.000000,7.000000,NaN,0.000000,NaN,NaN,NaN,NaN
50%,4.818874e+07,NaN,NaN,NaN,NaN,3.991500e+05,NaN,NaN,1997.000000,2.000000,3.000000,1828.000000,NaN,6969.600000,NaN,3.955000e+05,28.162615,-80.685962,0.000000,14.000000,NaN,0.000000,NaN,NaN,NaN,NaN
75%,1.041317e+08,NaN,NaN,NaN,NaN,5.250000e+05,NaN,NaN,2020.000000,3.000000,4.000000,2256.000000,NaN,9147.600000,NaN,5.230000e+05,28.342618,-80.639813,0.000000,28.000000,NaN,38.000000,NaN,NaN,NaN,NaN


---
Follow the **five** steps found in the **Data Detox** Towards Data Science article.
1) Check data types and nulls - DONE
2) Check for duplicate entries - DONE
3) Check for unnecessary white space
4) Check min and max values
5) Check for outliers

In [1118]:
df_forSale_trunc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2245 entries, 0 to 2244
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   zpid                          2245 non-null   int64  
 1   homeType                      2245 non-null   object 
 2   homeStatus                    2245 non-null   object 
 3   timeOnZillow.1                2245 non-null   object 
 4   price                         2245 non-null   float64
 5   streetAddress                 2245 non-null   object 
 6   zipcode                       2245 non-null   object 
 7   yearBuilt                     2245 non-null   int64  
 8   bedrooms                      2245 non-null   float64
 9   bathrooms                     2245 non-null   float64
 10  livingAreaValue               2245 non-null   float64
 11  livingAreaUnits               2245 non-null   object 
 12  lotAreaValue                  2245 non-null   float64
 13  lot

In [1119]:
df_recentlySold_trunc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5682 entries, 0 to 5681
Data columns (total 26 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   zpid                          5682 non-null   int64  
 1   homeType                      5682 non-null   object 
 2   homeStatus                    5682 non-null   object 
 3   dateSoldString                5682 non-null   object 
 4   timeOnZillow.1                5682 non-null   object 
 5   price                         5682 non-null   float64
 6   streetAddress                 5682 non-null   object 
 7   zipcode                       5682 non-null   object 
 8   yearBuilt                     5682 non-null   int64  
 9   bathrooms                     5682 non-null   float64
 10  bedrooms                      5682 non-null   float64
 11  livingAreaValue               5682 non-null   float64
 12  livingAreaUnits               5682 non-null   object 
 13  lot

In [1120]:
for column_ in columnNames_forSale:
    nullValueCount = df_forSale_trunc[column_].isnull().sum()
    print(f'Number of NaN values in {column_}: {nullValueCount}')

Number of NaN values in zpid: 0
Number of NaN values in homeType: 0
Number of NaN values in homeStatus: 0
Number of NaN values in timeOnZillow.1: 0
Number of NaN values in price: 0
Number of NaN values in streetAddress: 0
Number of NaN values in zipcode: 0
Number of NaN values in yearBuilt: 0
Number of NaN values in bedrooms: 0
Number of NaN values in bathrooms: 0
Number of NaN values in livingAreaValue: 0
Number of NaN values in livingAreaUnits: 0
Number of NaN values in lotAreaValue: 0
Number of NaN values in lotAreaUnits: 0
Number of NaN values in zestimate: 0
Number of NaN values in latitude: 0
Number of NaN values in longitude: 0
Number of NaN values in favoriteCount: 0
Number of NaN values in pageViewCount: 0
Number of NaN values in newConstructionType: 1869
Number of NaN values in resoFacts.associationFee: 0
Number of NaN values in listingSubType.isForeclosure: 0
Number of NaN values in resoFacts.isSeniorCommunity: 0
Number of NaN values in resoFacts.propertySubType: 0
Number of

In [1121]:
for column_ in columnNames_recentlySold:
    nullValueCount = df_recentlySold_trunc[column_].isnull().sum()
    print(f'Number of NaN values in {column_}: {nullValueCount}')

Number of NaN values in zpid: 0
Number of NaN values in homeType: 0
Number of NaN values in homeStatus: 0
Number of NaN values in dateSoldString: 0
Number of NaN values in timeOnZillow.1: 0
Number of NaN values in price: 0
Number of NaN values in streetAddress: 0
Number of NaN values in zipcode: 0
Number of NaN values in yearBuilt: 0
Number of NaN values in bathrooms: 0
Number of NaN values in bedrooms: 0
Number of NaN values in livingAreaValue: 0
Number of NaN values in livingAreaUnits: 0
Number of NaN values in lotAreaValue: 0
Number of NaN values in lotAreaUnits: 0
Number of NaN values in zestimate: 0
Number of NaN values in latitude: 0
Number of NaN values in longitude: 0
Number of NaN values in favoriteCount: 0
Number of NaN values in pageViewCount: 0
Number of NaN values in newConstructionType: 0
Number of NaN values in resoFacts.associationFee: 0
Number of NaN values in listingSubType.isForeclosure: 0
Number of NaN values in resoFacts.isSeniorCommunity: 0
Number of NaN values in

In [1122]:
# Check for duplicates
print(f'There are {df_forSale_trunc.price.loc[df_forSale_trunc.duplicated()].count()} duplicates in df_forSale_trunc.')
print(f'There are {df_recentlySold_trunc.price.loc[df_recentlySold_trunc.duplicated()].count()} duplicates in df_recentlySold_trunc.')

There are 0 duplicates in df_forSale_trunc.
There are 0 duplicates in df_recentlySold_trunc.


In [1123]:
# Check for outliers
# Currently there is one house built in 1884 in df_forSale_trunc. This would count as an outlier I think. Probably need to remove.

In [1124]:
# del df_forSale_trunc, df_recentlySold_trunc